### Data Ingestion to Vector DB pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

c:\Users\User\Desktop\RagFundamentals\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdfs inside the directory

def process_all_pdfs(pdf_directory):
    all_documents = []
    pdf_dir = Path(pdf_directory)

    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print("Processing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f"✅ Loaded {len(documents)} pages")
        except Exception as e:
            print(f"Error {e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")


Found 5 PDF files to process
Processing: {pdf_file.name}
✅ Loaded 1 pages
Processing: {pdf_file.name}
✅ Loaded 1 pages
Processing: {pdf_file.name}
✅ Loaded 2 pages
Processing: {pdf_file.name}
✅ Loaded 1 pages
Processing: {pdf_file.name}
✅ Loaded 1 pages

Total documents loaded: 6


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-02-07T11:26:34+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-02-07T11:26:34+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '..\\data\\pdf\\data_engineering_for_ai_dummy.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'data_engineering_for_ai_dummy.pdf', 'file_type': 'pdf'}, page_content='Data Engineering for AI Systems\nData Pipelines\nAutomated flows that ingest, clean, and transform data.\nETL vs ELT\nETL transforms before loading; ELT transforms after loading.\nData Quality\nAccuracy, completeness, and consistency are critical for ML systems.\nStorage Systems\nData lakes, warehouses, and object storage support AI workloads.\nImportance for RAG\nReliable pipelines ensure high-quality documents for retrieval.'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.repo

In [4]:
### Text splitting get into chunks

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performs"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs


In [5]:
chunks = split_documents(all_pdf_documents)
chunks

Split 6 documents into 7 chunks

Example chunk:
Content: Data Engineering for AI Systems
Data Pipelines
Automated flows that ingest, clean, and transform data.
ETL vs ELT
ETL transforms before loading; ELT transforms after loading.
Data Quality
Accuracy, co
Metadata: {'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-02-07T11:26:34+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-02-07T11:26:34+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '..\\data\\pdf\\data_engineering_for_ai_dummy.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'data_engineering_for_ai_dummy.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-02-07T11:26:34+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-02-07T11:26:34+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '..\\data\\pdf\\data_engineering_for_ai_dummy.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'data_engineering_for_ai_dummy.pdf', 'file_type': 'pdf'}, page_content='Data Engineering for AI Systems\nData Pipelines\nAutomated flows that ingest, clean, and transform data.\nETL vs ELT\nETL transforms before loading; ELT transforms after loading.\nData Quality\nAccuracy, completeness, and consistency are critical for ML systems.\nStorage Systems\nData lakes, warehouses, and object storage support AI workloads.\nImportance for RAG\nReliable pipelines ensure high-quality documents for retrieval.'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.repo

### Embedding and VectorStoreDB

In [6]:
import numpy as np
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity 

In [23]:
import sys, torch
print("python:", sys.executable)
print("torch:", torch.__version__)
print("torch file:", torch.__file__)


ImportError: cannot import name 'package' from partially initialized module 'torch' (most likely due to a circular import) (c:\Users\User\Desktop\RagFundamentals\.venv\Lib\site-packages\torch\__init__.py)

In [7]:
from sentence_transformers import SentenceTransformer# Embedding model

In [8]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name # initializing the model name
        self.model = None  
        self._load_model()  # loading the specialized model 

    def _load_model(self):
        """Load the sentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)  # loading the model
            print(f"Model loaded sucessfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")   # getting the embedding dimension
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_emeddings(self, texts: List[str]) -> np.ndarray:
        """
            Generate embeddings for a list of texts

            Args:
                texts: List of text strings to embed

            returns:
                numpy array of embeddings with shape (len(texts), embedding_dim)
        """ 
        if not self.model:
            raise ValueError("MOdel not loaded")
    
        print(f"Generating embedding for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    

## Initializing the embedding manager

embedding_manager = EmbeddingManager()
embedding_manager




Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 458.83it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded sucessfully. Embedding dimension: 384


### Vector Store

In [ ]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector Store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory) # creating a client that has the reference to the vectordb

            # Get or Create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"{e}")


    def add_document(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of Langchain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDB
        
